In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import swat
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
os.environ["CAS_CLIENT_SSL_CA_LIST"]="/home/ds/cascert.pem"

conn = swat.CAS("d-crd-datasci01big.dev.cyber.sas.com", 5570)

In [ ]:
conn.setsessopt('WH')

In [ ]:
out = conn.loadtable('wls_day_02_hr13.sashdat', caslib='WH')

In [ ]:
clause = "EventID = 4625"
where = clause.format(table="", not_missing="Is Not Missing")

conn.partition(table={"name": "wls_day_02_hr13", "where":where}, 
               casout={"name": "WLS_DAY02_HR13_FAILED"})

In [ ]:
failedFull = conn.CASTable('WLS_DAY02_HR13_FAILED')

In [ ]:
len(failedFull)

In [ ]:
failedFull.UserName.distinct()

In [ ]:
conn.loadactionset("fedsql")

In [ ]:
query = f"CREATE TABLE WLS_DAY02_HR13_FAILED_USERS AS SELECT UserName, count(UserName) as Failures \
          FROM WLS_DAY02_HR13_FAILED GROUP BY UserName"

conn.fedsql.execdirect(query)

In [ ]:
failedUsers = conn.CASTable('WLS_DAY02_HR13_FAILED_USERS').sort('Failures', ascending=False)

In [ ]:
failedUsers.head(10)

In [ ]:
failedUsers.FAILURES.summary()['Summary'].transpose()

In [ ]:
y_pos = np.arange(20)

plt.figure(figsize=(20,5))
plt.bar(y_pos, failedUsers.FAILURES.values[:20], align='center', alpha=0.5)
plt.xticks(y_pos, failedUsers.UserName.values[:20], rotation=45)
plt.ylabel('Failed Login Attempts')
plt.title('Top 20 Login Failures By Username')
plt.savefig('loginFailures.png', bbox_inches='tight')

plt.show()

In [ ]:
conn.close()